In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import sys
sys.path.append('..') #get root directory



Now that we know what we know which features we want to use and the format we want it to be in we can try out some different models, hoping to pick out some promising ones. The first thing that needs to be done is get the data pipeline going.

# Transformation Pipeline 
Now that the preprocessing steps are defined we can wrap all of this neatly into a Pipeline, allowing us to train and test various different models more efficiently.

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from src.features import logTransformer
from src.features import cbrtTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
standard_scaler = StandardScaler()
one_hot_encoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore') # Instead of a SciPy sparse matrix, we get a NumPy array

In [5]:
preprocess = ColumnTransformer([("onehot_cities", one_hot_encoder, ['city']),("log", logTransformer(), ['sqft_lot']),('cbrt', cbrtTransformer(), ['sqft_living', 'sqft_basement', 'sqft_above'])], remainder = 'passthrough')
pipeline = Pipeline([("preprocess", preprocess),('stdscaler', standard_scaler)])


In [6]:
from src.dataset import split_training_val_test
housing_data = pd.read_csv('../data/interim/housing_data_with_dropped_features')
housing_data.drop('Unnamed: 0', inplace= True, axis=1)
y = housing_data.iloc[:, : 1]
X = housing_data.iloc[:, 1: ]
X_train, X_val, X_test, y_train, y_val, y_test, train_df = split_training_val_test(X=X, y=y)
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,city
3204,3.0,1.75,1720,4080,1.0,0,0,4,960,760,Seattle
407,3.0,2.25,1445,1606,2.0,0,0,3,1300,145,Issaquah
42,2.0,1.00,850,6174,1.0,0,0,4,850,0,Shoreline
27,4.0,2.25,2200,11250,1.5,0,0,5,1300,900,Kirkland
3517,4.0,3.25,3060,3898,2.0,0,0,3,2300,760,Issaquah
...,...,...,...,...,...,...,...,...,...,...,...
3797,4.0,1.75,1420,5000,1.5,0,0,4,1420,0,Seattle
4025,3.0,2.50,2040,10086,2.0,0,0,3,2040,0,Redmond
955,4.0,2.75,2050,3960,1.0,0,0,4,1180,870,Seattle
3716,4.0,1.75,2130,5080,1.5,0,0,3,2130,0,Seattle


In [7]:
X_train_transformed_data = pipeline.fit_transform(X_train) 
X_train_transformed = pd.DataFrame(data=X_train_transformed_data, index = X_train.index, columns= pipeline.get_feature_names_out())
X_train_transformed.head()

,onehot_cities__city_Algona,onehot_cities__city_Auburn,onehot_cities__city_Bellevue,onehot_cities__city_Black Diamond,onehot_cities__city_Bothell,onehot_cities__city_Burien,onehot_cities__city_Carnation,onehot_cities__city_Clyde Hill,onehot_cities__city_Covington,onehot_cities__city_Des Moines,...,log__sqft_lot,cbrt__sqft_living,cbrt__sqft_basement,cbrt__sqft_above,remainder__bedrooms,remainder__bathrooms,remainder__floors,remainder__waterfront,remainder__view,remainder__condition
3204,-0.022135,-0.188406,-0.24987,-0.031311,-0.073594,-0.122109,-0.049544,-0.038358,-0.086024,-0.122109,...,-0.747605,-0.331192,1.265745,-1.162915,-0.428522,-0.499473,-0.956399,-0.076885,-0.298047,0.765143
407,-0.022135,-0.188406,-0.24987,-0.031311,-0.073594,-0.122109,-0.049544,-0.038358,-0.086024,-0.122109,...,-1.809191,-0.708019,0.395872,-0.570130,-0.428522,0.154115,0.902684,-0.076885,-0.298047,-0.672228
42,-0.022135,-0.188406,-0.24987,-0.031311,-0.073594,-0.122109,-0.049544,-0.038358,-0.086024,-0.122109,...,-0.275935,-1.729744,-0.784317,-1.384509,-1.534770,-1.479856,-0.956399,-0.076885,-0.298047,0.765143
27,-0.022135,-0.188406,-0.24987,-0.031311,-0.073594,-0.122109,-0.049544,-0.038358,-0.086024,-0.122109,...,0.407256,0.239909,1.384602,-0.570130,0.677726,0.154115,-0.026857,-0.076885,-0.298047,2.202514
3517,-0.022135,-0.188406,-0.24987,-0.031311,-0.073594,-0.122109,-0.049544,-0.038358,-0.086024,-0.122109,...,-0.799564,1.082850,1.265745,0.721655,0.677726,1.461291,0.902684,-0.076885,-0.298047,-0.672228


In [8]:
X_val_transformed_data = pipeline.transform(X_val)
X_val_transformed = pd.DataFrame(data= X_val_transformed_data, index = X_val.index, columns = pipeline.get_feature_names_out())

X_test_transformed_data = pipeline.transform(X_test)
X_test_transformed = pd.DataFrame(data = X_test_transformed_data, index = X_test.index, columns = pipeline.get_feature_names_out())

y_train_transformed = np.log(y_train) # doesn't need to go through a pipeline so no need to use logTransformer class
y_test_transformed = np.log(y_test)
y_val_transformed = np.log(y_val)

# Model Selection

Since this is a supervised learning problem and we want to predict a house price we will use a regression model. We'll try out a few: linear regression, suppor vector regression, and random forest regressor. 

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline


In [10]:
from scipy.stats import randint
linear_regression_param_grid = [{'linearregression__fit_intercept': [True, False], 'linearregression__positive': [True, False]}]

svr_params_grid = [{'svr__kernel': ['poly', 'linear'], 'svr__C': [40, 50, 60]},
          {'svr__kernel': ['poly'], 'svr__degree': [2,4,5]},
          {'svr__kernel': ['rbf, sigmoid'], 'svr__gamma': ['auto']},
           {'svr__kernel': ['linear'], 'svr__C' : [70,80,100]}]

random_forest_params_grid = [{'randomforestregressor__n_estimators': randint(100, 201), 'randomforestregressor__criterion': ['squared_error','absolute_error', 'friedman_mse', 'poisson']}]


In [11]:
linear_regression_pipeline = make_pipeline(pipeline, LinearRegression())
linear_regression_randomized_search = RandomizedSearchCV(linear_regression_pipeline, linear_regression_param_grid, scoring = 'neg_root_mean_squared_error', cv = 5, n_iter = 4)
linear_regression_randomized_search.fit(X_train, y_train_transformed)


/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('preprocess',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('onehot_cities',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse_output=False),
                                                                                                ['city']),
                                                                                               ('log',
                                                                                                logTransformer(),
                                                                                                ['sqft_lot']),
                                                                                               ('cbrt',
                                                                                                cbrtTransformer(),
                                                                                                ['sqft_living',
                                                                                                 'sqft_basement',
                                                                                                 'sqft_above'])])),
                                                              ('stdscaler',
                                                               StandardScaler())])),
                                             ('linearregression',
                                              LinearRegression())]),
                   n_iter=4,
                   param_distributions=[{'linearregression__fit_intercept': [True,
                                                                             False],
                                         'linearregression__positive': [True,
                                                                        False]}],
                   scoring='neg_root_mean_squared_error')

In [12]:
linear_regression_randomized_search_bestscore = -linear_regression_randomized_search.best_score_
linear_regression_randomized_search_bestparams = linear_regression_randomized_search.best_params_
print(linear_regression_randomized_search_bestparams)
print(linear_regression_randomized_search_bestscore)

{'linearregression__positive': False, 'linearregression__fit_intercept': True}
0.3018110804609648


In [13]:
svr_pipeline = make_pipeline(pipeline, SVR())
svr_randomized_search = RandomizedSearchCV(svr_pipeline, svr_params_grid, scoring = 'neg_root_mean_squared_error', cv = 5, n_iter = 10)
y_train_flattened = np.ravel(y_train_transformed) # svr excepts a 1d array not a column vector
svr_randomized_search.fit(X_train, y_train_flattened)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.f

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('preprocess',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('onehot_cities',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse_output=False),
                                                                                                ['city']),
                                                                                               ('log',
                                                                                                logTransformer(),
                                                                                                ['sqft_lot']),
                                                                                               ('cbrt',
                                                                                                cbrtTransformer(),
                                                                                                ['sqft_living',
                                                                                                 'sqft_basement',
                                                                                                 'sqft_above'])])),
                                                              ('stdscaler',
                                                               StandardScaler())])),
                                             ('svr', SVR())]),
                   param_distributions=[{'svr__C': [40, 50, 60],
                                         'svr__kernel': ['poly', 'linear']},
                                        {'svr__degree': [2, 4, 5],
                                         'svr__kernel': ['poly']},
                                        {'svr__gamma': ['auto'],
                                         'svr__kernel': ['rbf, sigmoid']},
                                        {'svr__C': [70, 80, 100],
                                         'svr__kernel': ['linear']}],
                   scoring='neg_root_mean_squared_error')

In [14]:
svr_randomized_search_bestscore = -svr_randomized_search.best_score_
svr_randomized_search_bestparams = svr_randomized_search.best_params_
print(svr_randomized_search_bestparams)
print(svr_randomized_search_bestscore)

{'svr__kernel': 'linear', 'svr__C': 40}
0.30069471777947926


In [15]:
random_forest_pipeline = make_pipeline(pipeline, RandomForestRegressor())
random_forest_randomized_search = RandomizedSearchCV(random_forest_pipeline, random_forest_params_grid, scoring = 'neg_root_mean_squared_error', cv = 5, n_iter = 10)
y_train_flattened = np.ravel(y_train_transformed)
random_forest_randomized_search.fit(X_train, y_train_flattened)



/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('preprocess',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('onehot_cities',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse_output=False),
                                                                                                ['city']),
                                                                                               ('log',
                                                                                                logTransformer(),
                                                                                                ['sqft_lot']),
                                                                                               ('cbrt',
                                                                                                cbrtTransformer(),
                                                                                                ['sqft_living',
                                                                                                 'sqft_basement',
                                                                                                 'sqft_above'])])),
                                                              ('stdscaler',
                                                               StandardScaler())])),
                                             ('randomforestregressor',
                                              RandomForestRegressor())]),
                   param_distributions=[{'randomforestregressor__criterion': ['squared_error',
                                                                              'absolute_error',
                                                                              'friedman_mse',
                                                                              'poisson'],
                                         'randomforestregressor__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1695b9be0>}],
                   scoring='neg_root_mean_squared_error')

In [16]:
random_forest_randomized_search_best_score = -random_forest_randomized_search.best_score_
random_forest_randomized_search_best_params = random_forest_randomized_search.best_params_
random_forest_randomized_search_best_estimator = random_forest_randomized_search.best_estimator_
print(random_forest_randomized_search_best_score)
print(random_forest_randomized_search_best_params)
print(random_forest_randomized_search_best_estimator)

0.3257078207210807
{'randomforestregressor__criterion': 'friedman_mse', 'randomforestregressor__n_estimators': 173}
Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('preprocess',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehot_cities',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['city']),
                                                                  ('log',
                                                                   logTransformer(),
                                                                   ['sqft_lot']),
                                                                  ('cbrt',
                                           

In [17]:
import joblib
import json 


The Joblib library will be used to store the results and fitted models to avoid having to rerun the code cells, helping to save time.

In [18]:
svr_info = {"svr_best_estimator" : svr_randomized_search_bestscore , "svr_best_params": svr_randomized_search_bestparams }
lr_info = {"lr_best_estimator" : linear_regression_randomized_search_bestscore, "lr_best_params": linear_regression_randomized_search_bestparams}
rfr_info = {"rfr_best_estimator": random_forest_randomized_search_best_score, "rfr_best_params" : random_forest_randomized_search_best_params}

In [21]:
with open("../models/svr_info.json", "w") as f:
  json.dump(svr_info, f, indent= 2)

with open("../models/lr_info.json", "w") as f:
  json.dump(lr_info, f, indent= 2)

with open("../models/rfr_info.json", "w") as f:
  json.dump(rfr_info, f, indent= 2)

In [20]:
joblib.dump(svr_randomized_search.best_estimator_, '../models/svr_model.joblib')
joblib.dump(linear_regression_randomized_search.best_estimator_, '../models/lr_model.joblib')
joblib.dump(random_forest_randomized_search_best_estimator, '../models/rfr_model.joblib')

['../models/rfr_model.joblib']